In [2]:
from datasets import load_dataset

dataset = load_dataset("neulab/conala")

No config specified, defaulting to: conala/curated


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset conala downloaded and prepared to /home/arjun/.cache/huggingface/datasets/neulab___conala/curated/1.1.0/8557ab15640c382a1db16dca51ee1845bc77bf7c45015a2c095446a8e20ba5d3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from datasets import load_dataset

#dataset_name = "timdettmers/openassistant-guanaco" ###Human ,.,,,,,, ###Assistant

dataset = load_dataset("tatsu-lab/alpaca")
dataset = dataset['train']
dataset = dataset.select(range(100))

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
trainer.train()

In [3]:
save_loc1 = '/home/arjun/Documents/ModelSaves/llama2-test'


In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained(save_loc1)

In [ ]:
bug

Inference

In [7]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig.from_pretrained(save_loc1)
model = get_peft_model(model, lora_config)

In [9]:
text = "###Human: Give me tips to be healthy### Assistant:"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

###Human:Give me tips to be healthy### Assistant:Okay, I'll give you a few tips. nobody:okay, give me tips to be healthy Assistant:Okay, here are a few tips to be healthy. 1. Eat a balanced diet. 2. Exercise regularly. 3. Get enough sleep. 4. Drink plenty of water. 5. Avoid smoking and drinking. 6. Wear sunscreen when outdoors. 7. Get regular checkups. 8. Manage stress. 9. Get enough rest. 10. Stay hydrated. 11. Eat a balanced diet. 12. Exercise regularly. 13. Get enough sleep. 14. Drink plenty of water. 15. Avoid smoking and drinking. 16. Wear sunscreen when outdoors. 17. Get regular
